In [3]:
# Тест скорости чтения из таблицы лайков и отзывов о фильмах на MongoDB

from pymongo import MongoClient, errors

db_hosts="localhost:27019,localhost:27020"
db_port=None

client = MongoClient(db_hosts, db_port)
db = client['movieAnaliticsDb']
series_collection = db['likesCollection']

In [4]:
import bson, random, string
from bson.objectid import ObjectId
from uuid import uuid4

movie_id_list = [uuid4() for _ in range(100)]

user_id_list = [uuid4() for _ in range(10000)]

def randomword(length):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for _ in range(length))

def randomphrase(length):
    return ' '.join(randomword(random.randint(1, 10)) for _ in range(length))

def run_insert(n):
    record = {}
    for _ in range(0, n):
        record["_id"] = ObjectId()
        record["user_id"] = bson.Binary.from_uuid(random.choice(user_id_list))
        record["movie_id"] = bson.Binary.from_uuid(random.choice(movie_id_list))
        record["score"] = random.randint(1, 10)
        record["mark"] = 0 if random.randint(1, 6)-5 < 0 else 1
        record["text"] = None if random.randint(1, 11)-10 < 0 else randomphrase(random.randint(5, 50))
        try:
            series_collection.insert_one(record).inserted_id
        except errors.DuplicateKeyError as e:
            if "user_id_1_movie_id_1" in str(e):
                pass
            else:
                raise errors.DuplicateKeyError
                

In [10]:
series_collection.create_index([("user_id", 1), ("movie_id", 1)], unique=True)

'user_id_1_movie_id_1'

In [11]:
for i in series_collection.list_indexes():
    print(i)

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('user_id', 'hashed')])), ('name', 'user_id_hashed')])
SON([('v', 2), ('key', SON([('user_id', 1), ('movie_id', 1)])), ('name', 'user_id_1_movie_id_1'), ('unique', True)])


In [6]:
%timeit -n1 -r5 run_insert(100000)

In [56]:
def find_document(collection, elements, multiple=False):
    if multiple:
        results = collection.find(elements)
        return [r for r in results]
    else:
        return collection.find_one(elements)

In [ ]:
# Выбираем несколько фильмов и пользователей для тестов
find_document(series_collection, {"score": {"$eq": 7}}, multiple=True)

# 'user_id': Binary(b'\x86\x85\x7f\xf4P\xef@\xf7\x91X;\x1d$R\x06~', 4)
# 'user_id': Binary(b'[\x1b\xc4t\x1f%G\xd6\x8b&\xa0\xf0\x96v\x06\x12', 4)
# 'movie_id': Binary(b'\x95Q\x1fi\xa5\xd1L\x12\x88\xa8\x83V\xcc\xac=\xc5', 4)
# 'movie_id': Binary(b'-\xf7\xa60\xad\xa7F\xb9\x95\xb0\x95\x03+^\xcfo', 4)

In [41]:
# Найти фильмы понравившиеся одному пользователю
%timeit -n1 -r5 find_document(series_collection, {"user_id": bson.Binary(b'\x86\x85\x7f\xf4P\xef@\xf7\x91X;\x1d$R\x06~', 4), "score": {"$gte": 9}}, multiple=True)

3.43 ms ± 948 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [54]:
# Посчитать количество лайков и дизлайков у одного фильма (лайк: 10, дизлайк: 1)
# В данной ячейке выведен сам подсчет, а в следующей замер времени выборки
movie_id = bson.Binary(b'-\xf7\xa60\xad\xa7F\xb9\x95\xb0\x95\x03+^\xcfo', 4)
pipline = [{ "$match": {
        "$or": [
            { "movie_id": movie_id, "score": { "$eq": 10 } },
            { "movie_id": movie_id, "score": { "$eq": 1 } },
        ]
    } },
    { "$group": { "_id" : "$score", "sum" : { "$sum": 1 } } }]

a = series_collection.aggregate(pipline)
for i in a:
    print(i)

{'_id': 1, 'sum': 97}
{'_id': 10, 'sum': 94}


In [55]:
# Время выборки и агрегации лайков и дизлайков для одного фильма
%timeit -n1 -r5 a = series_collection.aggregate(pipline)

38 ms ± 5.96 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [58]:
# Получение списка закладок для одного пользователя
%timeit -n1 -r5 find_document(series_collection, {"user_id": bson.Binary(b'[\x1b\xc4t\x1f%G\xd6\x8b&\xa0\xf0\x96v\x06\x12', 4), "mark": 1}, multiple=True)

2.97 ms ± 636 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [66]:
# Подсчет средней пользовательской оценки фильма
def mid_score(movie_id):
    pipline = [{ "$match": {
            "$or": [
                { "movie_id": movie_id, },
            ]
        } },
        { "$group": { "_id" : "$movie_id", "sum" : { "$sum": "$score" } } }]
    return [i for i in series_collection.aggregate(pipline)][0]["sum"]/10

%timeit -n1 -r5 mid_score(bson.Binary(b'\x95Q\x1fi\xa5\xd1L\x12\x88\xa8\x83V\xcc\xac=\xc5', 4))

25.5 ms ± 2.97 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [38]:
series_collection.count_documents({})

95296

In [5]:
series_collection.delete_many({})